In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2 as cv
import os, shutil
from pathlib import Path

from skimage import feature as ft
import cv2
import math
import re


In [3]:
model = tf.keras.applications.MobileNetV2(include_top=False,
weights="imagenet", input_shape=(224, 224, 3))

14536120/14536120 [==============================] - 76s 5us/step

In [1]:
# 获得该文件夹下所有jpg图片路径
f=open('./flowers/train.txt', encoding="utf-8")
files = f.readlines()  # 直接将文件中按行读到list里，效果与方法2一样
f.close()


In [2]:
i = 0
train_label = []
for _ in files:
    elements = re.split(r'[-\s,;!?]+', _)
    files[i] = elements[0]
    train_label.extend(elements[1])
    i += 1
i = 0

NameError: name 're' is not defined

##### opencv读取图像，并将图像大小 resize 为（224，224），以匹配模型输入层的大小以进行特征提取。图像数组转换为 float32 类型并reshape，然后做归一化。



In [12]:
# opencv读取图像 并resize为（224，224）
images = [cv.resize(cv.imread(str(file)), (224, 224)) for file in files]
paths = [file for file in files]
# 图像数组转换为float32类型并reshape  然后做归一化
images = np.array(np.float32(images).reshape(len(images), -1) / 255)

##### 加载预训练模型 MobileNetV2 来实现图像分类

In [13]:
# 加载预先训练的模型MobileNetV2来实现图像分类

predictions = model.predict(images.reshape(-1, 224, 224, 3))
pred_images = predictions.reshape(images.shape[0], -1)

135/135 [==============================] - 143s 962ms/step

##### k-means聚类算法

In [15]:
k = 5   # 5个类别
# K-Means聚类
kmodel = KMeans(n_clusters=k,  random_state=888)
kmodel.fit(pred_images)
kpredictions = kmodel.predict(pred_images)
print(kpredictions)   # 预测的类别
# 0：dog    1：cat

[4 4 2 ... 2 1 2]


In [16]:
kpredictions

array([4, 4, 2, ..., 2, 1, 2])

In [17]:
pred_images

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.32531357, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [18]:
for i in flowers:
    os.mkdir(r"./Result/" + str(i))



In [19]:
# 复制文件，保留元数据 shutil.copy2('来源文件', '目标地址')
for i in range(len(paths)):
    if kpredictions[i] == 0:
        shutil.copy2(paths[i], r"./Result/daisy")
    elif kpredictions[i] == 1:
        shutil.copy2(paths[i], r"./Result/dandelion")
    elif kpredictions[i] == 2:
        shutil.copy2(paths[i], r"./Result/rose")
    elif kpredictions[i] == 3:
        shutil.copy2(paths[i], r"./Result/sunflower")
    elif kpredictions[i] == 4:
        shutil.copy2(paths[i], r"./Result/tulip")

In [20]:
print(len(images))
len(pred_images)

4317


4317

In [33]:
i = 0
flowers = ['daisy','dandelion','rose','sunflower','tulip']
for flower in flowers:
    cat = []
    p = Path(r"./flowers/"+str(flower))
    lists = list(p.glob("**/*.jpg"))
    l = kpredictions[i:i+len(lists)]
    most_ele = np.argmax(np.bincount(l))

    n=0
    for i in l:
        if i == most_ele:
            n +=1

    print('most:'+str(most_ele)+':'+str(float(n/len(lists))))
    i += len(lists)

most:4:0.6361256544502618
most:0:0.5988593155893536
most:0:0.576530612244898
most:0:0.592087312414734
most:0:0.5772357723577236


In [27]:
kpredictions[0:5]


array([4, 4, 2, 4, 1])